In [1]:
%load_ext watermark
%watermark

2019-01-03T23:54:35-06:00

CPython 3.7.1
IPython 7.2.0

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.15.0-43-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12, 12]
np.random.seed(42)

# KNN - K vecinos más próximos

Vamos a ver como vamos a usar el algoritmo KNN en scikit-learn.

El algoritmo KNN se puede usar tanto en problemas de clasificación (con el estimador [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)) como en problemas de regresión (con el estimador [KNeighborsRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor))

### Cargamos los datos

Para este ejemplo vamos a usar el dataset [CSM (Conventional and Social Media Movies)](https://archive.ics.uci.edu/ml/datasets/CSM+%28Conventional+and+Social+Media+Movies%29+Dataset+2014+and+2015) que contiene información de la popularidad en redes sociales de distintas películas así como las ventas en taquilla.

In [47]:
pelis = pd.read_csv("data/datos_peliculas.csv")
pelis.shape

(231, 12)

In [48]:
pelis.head()

,pelicula,año,ratings,genero,ventas,presupuesto,secuela,vistas_youtube,positivos_youtube,negativos_youtube,comentarios,seguidores_agregados
0,13 Sins,2014,6.3,8,9130,4000000.0,1,3280543,4632,425,636,1120000.0
1,22 Jump Street,2014,7.1,1,192000000,50000000.0,2,583289,3465,61,186,12350000.0
2,3 Days to Kill,2014,6.2,1,30700000,28000000.0,1,304861,328,34,47,483000.0
3,300: Rise of an Empire,2014,6.3,1,106000000,110000000.0,2,452917,2429,132,590,568000.0
4,A Haunted House 2,2014,4.7,8,17300000,3500000.0,2,3145573,12163,610,1082,1923800.0


Vamos a eliminar el título de las películas

In [49]:
pelis = pelis.drop("pelicula", axis=1)

Vamos a usar este dataset para probar KNN en clasificación y en regresión

In [50]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split

### KNN para problemas de clasificación

Probamos KNN para clasificación, en concreto vamos a suponer que queremos predecir el género de una película en función de su popularidad.

In [51]:
from sklearn.metrics import f1_score

variable_objetivo_clasificacion = "genero"
variables_independientes_clasificacion = pelis.drop(
    variable_objetivo_clasificacion, axis=1).columns

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_clasificacion],
    pelis[variable_objetivo_clasificacion], test_size=0.20)

In [53]:
KNeighborsClassifier?

Los parámetros más importantes a la hora de usar `KNeighborsClasifier` son:

- **n_neighbors**: El valor de K, es decir el número de vecinos que considerar a la hora de asignar una clase.
- **weights**: A la hora de votar, que importancia dar a los vecinos. Si elegimos `auto` asigna la misma importancia a todos los vecinos. Si elegimos `distance` asigna importancia a los vecinos en función de la distancia de los vecinos al punto a clasificar
- **metric**: La métrica a la hora de medir la distancia entre los puntos. Si se usa distancia de Minkowsky se puede elegir p con el parámetro `p`, que por defecto es 2 (lo que computa la distancia euclidiana).

En este caso en particular sabemos que valor elegir de K, ya que podemos asumir que el número de categorías del dataset es el total de categorías de películas del dataset de entrenamiento.

In [54]:
k_categorias = len(y_train.unique())
k_categorias

10

In [55]:
clasificador_knn = KNeighborsClassifier(n_neighbors=5, 
                                        weights="uniform")

clasificador_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [56]:
preds = clasificador_knn.predict(X_test)
f1_score(y_test, preds, average="micro")

0.3404255319148936

Si ahora entrenamos el estimador con el argumento de pesos `weights="distance"`, vemos que funciona de forma ligeramente mejor.

In [57]:
clasificador_knn = KNeighborsClassifier(n_neighbors=10, 
                                        weights="distance")

clasificador_knn.fit(X_train, y_train)

preds = clasificador_knn.predict(X_test)
f1_score(y_test, preds, average="micro")

0.3617021276595745

Podemos usar el método `kneighbors` para devolver los k vecinos de un punto en concreto

In [58]:
X_test.iloc[0]

año                         2014.0
ratings                        6.2
ventas                  50500000.0
presupuesto             60000000.0
secuela                        5.0
vistas_youtube           3320754.0
positivos_youtube           4322.0
negativos_youtube            347.0
comentarios                 1105.0
seguidores_agregados      147000.0
Name: 59, dtype: float64

In [59]:
distancia, indice = clasificador_knn.kneighbors(
    [X_test.iloc[0]], n_neighbors=1)
distancia, indice

(array([[1405048.94660163]]), array([[113]]))

In [60]:
X_train.iloc[indice[0]]

,año,ratings,ventas,presupuesto,secuela,vistas_youtube,positivos_youtube,negativos_youtube,comentarios,seguidores_agregados
114,2014,6.8,50800000,60000000.0,1,2545852,3964,378,554,1280000.0


Vemos que el vecino más cercano es similar.

### KNN para problemas de regresión

Vamos a utilizar ahora el algoritmo KNN para un problema de regresión, KNN funciona igual para hacer regresiones, simplemente que en vez de una votación donde la clase más común entre los vecinos más próximos es la elegida, se hace una interpolación de los valores de la variable numérica objetivo de los vecinos.

en concreto vamos a estimar las ventas de entradas en taquilla de una película en función de su popularidad online y presupuesto.

In [61]:
from sklearn.metrics import mean_squared_error

variable_objetivo_regresion = "ventas"
variables_independientes_regresion = pelis.drop(
    variable_objetivo_regresion, axis=1).columns

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_regresion],
    pelis[variable_objetivo_regresion], test_size=0.20)

Usamos la implementación en sklearn `KNeighborsRegressor` para problemas de regresión. Tiene los mismos hiperparámetros que `NeighborsClassifier`.

In [26]:
KNeighborsRegressor?

In [63]:
regresor_knn = KNeighborsRegressor(n_neighbors=10, weights="distance")

regresor_knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='distance')

In [64]:
preds = regresor_knn.predict(X_test)
preds

array([1.27165919e+07, 2.61092729e+07, 9.96009544e+07, 8.87430669e+07,
       1.35091554e+08, 4.03106692e+07, 1.36246100e+08, 1.77072442e+08,
       4.70123782e+07, 2.42447638e+07, 1.15911189e+07, 7.98148771e+07,
       9.85475214e+06, 6.01894949e+07, 3.84321187e+07, 8.93856167e+06,
       4.24257527e+07, 2.26155080e+08, 3.11403228e+07, 9.54842776e+07,
       3.91773230e+07, 9.56558550e+07, 9.31405388e+07, 1.20724601e+08,
       6.31634738e+07, 2.01749107e+07, 5.04027022e+07, 3.64992259e+07,
       7.43292302e+07, 8.98069307e+07, 2.69466479e+07, 2.59234835e+07,
       9.79738880e+06, 3.29277419e+07, 6.06033737e+07, 4.58436357e+07,
       2.18923652e+08, 5.74621113e+07, 1.58319986e+08, 2.32477091e+08,
       3.41653495e+07, 1.31483670e+07, 1.33858780e+08, 2.92190586e+07,
       9.50689200e+06, 5.66691142e+07, 4.08123083e+07])

In [65]:
np.sqrt(np.abs(mean_squared_error(y_test, preds)))

52494085.058179684

Ahora vemos el funcionamiento en validación cruzada tanto del clasificador como el regresor

In [66]:
from sklearn.model_selection import cross_val_score

In [68]:
error_validacion_cruzada_clasificacion = np.sqrt(np.abs(
    cross_val_score(KNeighborsClassifier(n_neighbors=k_categorias,
                                         weights="distance"), 
                X=pelis[variables_independientes_clasificacion],
               y=pelis[variable_objetivo_clasificacion], 
               scoring="f1_micro"
        ).mean()
      )
)
print("La puntuación F1 de KNN para clasificacion en este dataset es {:.2f}".format(
    error_validacion_cruzada_clasificacion))

La puntuación F1 de KNN para clasificacion en este dataset es 0.54


/home/erdvillegas/anaconda3/envs/Datascientist/lib/python3.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [69]:
error_validacion_cruzada_regresion = np.sqrt(np.abs(
    cross_val_score(KNeighborsRegressor(weights="distance"), 
                X=pelis[variables_independientes_regresion],
               y=pelis[variable_objetivo_regresion], 
               scoring="neg_mean_squared_error"
        ).mean()
      )
)
print("El error RMSE de KNN para regresión en este dataset es {:.2f}$".format(
    error_validacion_cruzada_regresion))

El error RMSE de KNN para regresión en este dataset es 69488698.28$
